In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/temporal')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Extra_and_Unused')
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

Running local


In [2]:
# from __future__ import print_function
# from __future__ import print_function
import os
import numpy as np

import datetime
import gc

from scipy import signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel # For AE
# from PosAE_training_functions import posautoencoding_m1 as AE, TestModel, TrainModel # For PosAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta

# wandb.login()

cuda


In [5]:
def featuring_autoencoders(dataset, date_format, model, len_features=None, save=True, identifier=None):
    training_loader = DataLoader(dataset, batch_size=1)
    iterator = iter(training_loader)
    testing = TestModel(model, iterator, device=torch.device("cuda"))

    training_recorder_list = []
    training_hour_list = []
    training_minute_list = []
    delete_samples = []
    training_path_samples = []
    training_samples_list_torch = []
    
    if len_features == None:
        len_features = len(training_loader)
    else:
        len_features = len_features
        
    batch = int(len_features*0.1)
    
    for id in range(len_features):
        if (id+1)% batch == 0:
            print(f"id: {id + 1} of {len_features}")
        try:
            originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
        except:
            print(f"error id: {id}")
            delete_samples.append(id)
            continue

    #     encodings_size = encodings[0].shape
        encodings = encodings.to("cuda").detach()
        encodings = encodings.reshape(encodings.shape[0],
                                    encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
        training_samples_list_torch.append(encodings)
        label["recorder"]=np.repeat(label["recorder"][0][0],5)
#         training_recorder_list.append(label["recorder"])
#         training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
#         training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))


        path = np.asarray(path, dtype="U32")
        path = np.repeat(path, 5)
        indexer = ["_1","_2","_3","_4","_5"]
        indexer = np.asarray(indexer)
        # path = path.astype("float64")
        for i in range(len(path)):
            path[i] = label["recorder"][0] + "_" + path[i] + indexer[i]

        training_path_samples.append(path)
    
#     training_recorder_list = torch.cat(training_recorder_list,dim=0)
#     training_hour_list = torch.cat(training_hour_list,dim=0)
#     training_minute_list = torch.cat(training_minute_list,dim=0)
    training_samples_list_torch = torch.cat(training_samples_list_torch, dim=0)
    
    if save == True:
    
        if "filters" in dataset.kwargs.keys():
            for value in dataset.filters.values():
                date_format = f"{date_format}_{value}" 
        if identifier != None:
            date_format = f"{date_format}_{identifier}"
        else:
            pass

        torch.save(training_path_samples, f"temporal_zamuro/Features/epochs_50/AE_test_path_samples_{date_format}.pth")
        torch.save(training_samples_list_torch, f"temporal_zamuro/Features/epochs_50/AE_features_{date_format}.pth")
#         training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
#         torch.save(training_labels_list, f"temporal_zamuro/Features/epochs_50/AE_labels_{date_format}.pth")
#         torch.save(delete_samples, f"temporal_zamuro/Features/epochs_50/AE_test_corrupted_samples_list_{date_format}.pth")
        
    else:
        pass

models = os.listdir(f"{root}/Zamuro/temporal_zamuro/models/model_AE_Standarized_batch_size_20_num_hiddens_64__month_7_day_25_50_epochs")
sorted(models)
for i, model in enumerate(models):
    split = model.split("_")[-3::]
    print(split)
    os.rename(f"{root}/Zamuro/temporal_zamuro/models/model_AE_Standarized_batch_size_20_num_hiddens_64__month_7_day_25_50_epochs/{model}", f"{root}/Zamuro/temporal_zamuro/models/model_AE_Standarized_batch_size_20_num_hiddens_64__month_7_day_25_50_epochs/{split[0]}_{split[1]}_{split[2]}")


In [ ]:
models = os.listdir(f"{root}/Zamuro/temporal_zamuro/models/model_AE_Standarized_batch_size_20_num_hiddens_64__month_7_day_25_50_epochs")

In [18]:
date_format = "month_7_day_25_50"
models = ["epoch_50_training.pth", "epoch_40_training.pth", "epoch_30_training.pth", "epoch_20_training.pth", "epoch_10_training.pth"]
epochs = [30]
for i, epoch in enumerate(epochs):
    model_name = f"{root}/Zamuro/temporal_zamuro/models/model_AE_Standarized_batch_size_20_num_hiddens_64__month_7_day_25_50_epochs/epoch_{epoch}_training.pth"
    model = AE(num_hiddens=64).to(device)
    model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

    import os
#     folders = os.listdir(f"Complementary_Files/Audios_Zamuro/")

    for folder in folders:
        print(folder)
        fold = folder.split(".")[0]        
        filters = {"rain_FI": "NO"}
        dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                             dataframe_path=f"Complementary_Files/Audios_Zamuro/{folder}",
                             audio_length=12, ext="wav",
                             win_length=1028, filters=filters)
        featuring_autoencoders(dataset, f"{date_format}_rain", model=model, len_features=None, save=True, identifier=f"{fold}_epoch_{epoch}")

RZUB05.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 72 of 725
id: 144 of 725
id: 216 of 725
id: 288 of 725
id: 360 of 725
id: 432 of 725
id: 504 of 725
id: 576 of 725
id: 648 of 725
id: 720 of 725
RZUB11.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 67 of 675
id: 134 of 675
id: 201 of 675
id: 268 of 675
id: 335 of 675
id: 402 of 675
id: 469 of 675
id: 536 of 675
id: 603 of 675
id: 670 of 675
RZUB01.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 88 of 886
id: 176 of 886
id: 264 of 886
id: 352 of 886
id: 440 of 886
id: 528 of 886
id: 616 of 886
id: 704 of 886
id: 792 of 886
id: 880 of 886
RZUC03.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 68 of 682
id: 136 of 682
id: 204 of 682
id: 272 of 682
id: 340 of 682
id: 408 of 682
id: 476 of 682
id: 544 of 682
id: 612 of 682
id: 680 of 682
RZUF03.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 68 of 682


### Features by folder

In [ ]:
# folders = ["Audios_Jaguas"]
for folder in folders:
    print(folder)
    filters = {"rain_FI": "NO"}
    dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                         dataframe_path=f"Complementary_Files/Audios_Zamuro/{folder}",
                         audio_length=12, ext="wav",
                         win_length=1028, filters=filters)
    featuring_autoencoders(dataset, f"{date_format}_rain", model=model, len_features=None, save=True, identifier=f"{folder}")

### All features

In [ ]:
filters = {"rain_FI": "NO"}
dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                     dataframe_path="Complementary_Files/zamuro_audios.csv",
                     audio_length=12, ext="wav",
                     win_length=1028, filters=filters)

len_features=None
save=True
identifier=None

training_loader = DataLoader(dataset, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))

training_recorder_list = []
training_hour_list = []
training_minute_list = []
delete_samples = []
training_path_samples = []
training_samples_list_torch = []

if len_features == None:
        len_features = len(training_loader)
else:
    len_features = len_features
        
batch = int(len_features*0.1)

for id in range(len_features):
#     if (id+1)%3 == 0:
#         break
    if (id+1)% batch == 0:
        print(f"id: {id + 1} of {len_features}")
        
(originals, reconstructions, encodings, label, loss, path) = testing.reconstruct()
#         except:
#             print(f"error id: {id}")
#             delete_samples.append(id)
#             continue

    #     encodings_size = encodings[0].shape
encodings = encodings.to("cuda").detach()
encodings = encodings.reshape(encodings.shape[0],
                            encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
training_samples_list_torch.append(encodings)
label["recorder"]=np.repeat(label["recorder"][0][0],5)
training_recorder_list.append(label["recorder"])
training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))



In [ ]:
model_type = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 9
hour = 4
month = 6
folder = "AE_rain"
date_format = f"day_{day}_hour_{hour}"
y = torch.load(f"temporal_zamuro/Features/{folder}/AE_labels_{date_format}_rain_NO_RZUA01.pth",  map_location=torch.device('cpu'))
X = torch.load(f"temporal_zamuro/Features/{folder}/AE_features_{date_format}_rain_NO_RZUA01.pth",  map_location=torch.device('cpu'))
path = torch.load(f"temporal_zamuro/Features/{folder}/AE_test_path_samples_{date_format}_rain_NO_RZUA01.pth",  map_location=torch.device('cpu'))
path_flat = [item for sublist in path for item in sublist]
path_flat = np.asarray(path_flat)